In [1]:
### use the funtion 
from mymodule import check_df, find_categorical_columns, binary_columns_find
from sklearn.model_selection import cross_val_score, GridSearchCV, validation_curve
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
### 1. Import the necessary libraries 
### Importieren Sie die notwendigen Bibliotheken 

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import cross_val_score, GridSearchCV, validation_curve
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')


In [4]:
### 2 .Print first 5 row of the dataset
### Geben Sie die ersten 5 Zeilen des Datensatzes aus.

In [5]:
df=pd.read_csv('Salary Data.csv')

In [6]:
df.head()

,Experience Years,Salary
0,1.1,39343
1,1.2,42774
2,1.3,46205
3,1.5,37731
4,2.0,43525


In [7]:
### 1. Split the data into four pieces: train_x, test_x, train_y, and test_y . (The target value is the 'Salary')
## Teilen Sie die Daten in vier Teile auf: train_x, test_x, train_y und test_y. ('Der Zielwert ist die 'Salary')

In [8]:
X = df.drop('Salary', axis=1)
y = df[["Salary"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=99)

In [9]:
### Train XGBoost with below parameters  model 
from xgboost import XGBRegressor 

In [10]:
xgb_params = {
    'n_estimators': 50,
    'max_depth': 2,
    'learning_rate': 0.1,
    'subsample': 0.8
}
xgb_model = XGBRegressor(random_state=99, **xgb_params)
xgb_model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=2, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=50, n_jobs=None,
             num_parallel_tree=None, random_state=99, ...)

In [11]:
### Activate the code below and look at the model's score ( use cross_val_score with 3  )
### Aktiviere den unten stehenden Code und schaue dir die Punktzahl des Modells an (Verwende cross_val_score mit 3)

In [12]:
xgb_cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=3, scoring='r2')
y_pred_xgb = xgb_model.predict(X_test)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
xgb_train_score = xgb_model.score(X_train, y_train)
xgb_test_score = xgb_model.score(X_test, y_test)
mae = mean_absolute_error(y_test, y_pred_xgb)

In [13]:
print(f"XGBRegressor Cross-Validation Scores: {xgb_cv_scores}")
print(f"XGBRegressor Mean CV R^2 Score: {xgb_cv_scores.mean()}")
print(f"XGBRegressor Train R^2 Score: {xgb_train_score}")
print(f"XGBRegressor Test R^2 Score: {xgb_test_score}")
print(f"XGBRegressor  Test Mean Squared Error (MSE): {mse_xgb}")
print(f"XGBRegressor Test Root Mean Squared Error (RMSE): {np.sqrt(mse_xgb)}")
print(f"XGBRegressor Test Mean Absolute Error  (mae): {mae}")

XGBRegressor Cross-Validation Scores: [0.92682241 0.90814765 0.94307391]
XGBRegressor Mean CV R^2 Score: 0.9260146598595282
XGBRegressor Train R^2 Score: 0.9913379919105907
XGBRegressor Test R^2 Score: 0.912240131916928
XGBRegressor  Test Mean Squared Error (MSE): 25979786.135391235
XGBRegressor Test Root Mean Squared Error (RMSE): 5097.036995685948
XGBRegressor Test Mean Absolute Error  (mae): 3472.63671875


In [14]:
### Train the model with CatBoostRegressor, using the parameters below.
### Trainiere das Modell mit CatBoostRegressor unter Verwendung der untenstehenden Parameter.

In [15]:
from catboost import CatBoostRegressor

In [22]:
cat_params = {
    'iterations': 50,
    'depth': 2,
    'learning_rate': 0.1,
    'verbose': False
}
cat_model = CatBoostRegressor(random_state=99, **cat_params)
cat_cv_scores = cross_val_score(cat_model, X_train, y_train, cv=3, scoring='r2')
cat_model.fit(X_train, y_train)

In [17]:
### Check the model's scores
###Überprüfe die Punktzahlen des Modells.

In [36]:
cat_model.fit(X_train, y_train)
y_pred_cat = cat_model.predict(X_test)
cat_model_train_score = cat_model.score(X_train, y_train)
cat_model_test_score = cat_model.score(X_test, y_test)
mse_cat = mean_squared_error(y_test, y_pred_cat)
mae_cat = mean_absolute_error(y_test, y_pred_cat)

In [37]:
print(f"CatBoostRegressor Cross-Validation Scores: {cat_cv_scores}")
print(f"CatBoostRegressor Mean CV R^2 Score: {cat_cv_scores.mean()}")
print(f"CatBoostRegressor Train R^2 Score: {cat_model_train_score}")
print(f"CatBoostRegressor Test R^2 Score: {cat_model_test_score}")
print(f"CatBoostRegressor Mean Squared Error (MSE): {mse_cat}")
print(f"CatBoostRegressor Test Root Mean Squared Error (RMSE): {np.sqrt(mse_cat)}")
print(f"CatBoostRegressor Test Mean Absolute Error  (mae): {mae_cat}")

CatBoostRegressor Cross-Validation Scores: [0.92711246 0.4570604  0.98726005 0.94801106 0.95037116]
CatBoostRegressor Mean CV R^2 Score: 0.8539630264420912
CatBoostRegressor Train R^2 Score: 0.9782527553490439
CatBoostRegressor Test R^2 Score: 0.9023322085390857
CatBoostRegressor Mean Squared Error (MSE): 28912854.928960208
CatBoostRegressor Test Root Mean Squared Error (RMSE): 5377.0675027342
CatBoostRegressor Test Mean Absolute Error  (mae): 4631.735372867324
